## Import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
import tensorflow as tf

import cv2

from sklearn.model_selection import train_test_split

from keras.applications import Xception, VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

## Data Preparation

In [2]:
labels = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']
img_size = 256
data_dir = '/kaggle/input/plant-seedlings-classification/'
train_dir = data_dir + 'train/'
test_dir = data_dir + 'test/'
sample_submission = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')

## Training Data

In [3]:
train = []
for label_id, label in enumerate(labels):
    for file in os.listdir(os.path.join(train_dir, label)):
        train.append(['{}/{}'.format(label, file), label_id, label])
train = pd.DataFrame(train, columns=['file', 'label_id', 'label'])
train.head()

,file,label_id,label
0,Black-grass/429a48df1.png,0,Black-grass
1,Black-grass/3b7266ac3.png,0,Black-grass
2,Black-grass/13a7f553a.png,0,Black-grass
3,Black-grass/42336b187.png,0,Black-grass
4,Black-grass/775735fb9.png,0,Black-grass


## Image Pre-processing (Masking/Segmentation/Sharpening)

In [4]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [5]:
train_data = []

for i in range(len(train)):
    img = cv2.imread(os.path.join(train_dir, train['file'][i]))
    img = cv2.resize(img,(img_size,img_size))
    img = segment_plant(img)
    img = sharpen_image(img)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    train_data.append(np.array(img))

train_data = np.array(train_data)

### Label Encoding

In [6]:
train_label = train['label_id']
train_label =  to_categorical(train_label, 12)

## Train/Test set split

In [7]:
x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size = 0.1, random_state=10)

## Build VGG19 Model

In [8]:
base_model = VGG19(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
    
#compling and show model
model.compile(loss = "categorical_crossentropy", optimizer = 'Adadelta', metrics=["accuracy"])
model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_______________________________________________________

### Image Data Generator

In [9]:
datagen = ImageDataGenerator(rotation_range=180,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True)

datagen.fit(x_train)

## Fit Model

In [10]:
batch_size = 16 

In [12]:
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data = (x_val,y_val),
          steps_per_epoch = len(x_train)//batch_size,
          epochs = 100,
          verbose = 1)
model.save_weights("/kaggle/working/VGG19.h5")

Epoch 1/100
267/267 [==============================] - 90s 336ms/step - loss: 2.9202 - accuracy: 0.0946 - val_loss: 2.4310 - val_accuracy: 0.0947
Epoch 2/100
267/267 [==============================] - 86s 324ms/step - loss: 2.5298 - accuracy: 0.1134 - val_loss: 2.3375 - val_accuracy: 0.2568
Epoch 3/100
267/267 [==============================] - 89s 332ms/step - loss: 2.4177 - accuracy: 0.1463 - val_loss: 2.2342 - val_accuracy: 0.2947
Epoch 4/100
267/267 [==============================] - 87s 327ms/step - loss: 2.3536 - accuracy: 0.1740 - val_loss: 2.0626 - val_accuracy: 0.2800
Epoch 5/100
267/267 [==============================] - 88s 328ms/step - loss: 2.2762 - accuracy: 0.2099 - val_loss: 2.0162 - val_accuracy: 0.3326
Epoch 6/100
267/267 [==============================] - 88s 329ms/step - loss: 2.2067 - accuracy: 0.2334 - val_loss: 1.9167 - val_accuracy: 0.3789
Epoch 7/100
267/267 [==============================] - 87s 326ms/step - loss: 2.1435 - accuracy: 0.2707 - val_loss: 1.7863 -

Epoch 57/100
267/267 [==============================] - 87s 327ms/step - loss: 0.7152 - accuracy: 0.7671 - val_loss: 0.3746 - val_accuracy: 0.8800
Epoch 58/100
267/267 [==============================] - 87s 327ms/step - loss: 0.7025 - accuracy: 0.7668 - val_loss: 0.3647 - val_accuracy: 0.8779
Epoch 59/100
267/267 [==============================] - 87s 326ms/step - loss: 0.6847 - accuracy: 0.7734 - val_loss: 0.3521 - val_accuracy: 0.8842
Epoch 60/100
267/267 [==============================] - 88s 330ms/step - loss: 0.6834 - accuracy: 0.7786 - val_loss: 0.3455 - val_accuracy: 0.8821
Epoch 61/100
267/267 [==============================] - 90s 336ms/step - loss: 0.6563 - accuracy: 0.7805 - val_loss: 0.4514 - val_accuracy: 0.8589
Epoch 62/100
267/267 [==============================] - 88s 328ms/step - loss: 0.6760 - accuracy: 0.7781 - val_loss: 0.3843 - val_accuracy: 0.8716
Epoch 63/100
267/267 [==============================] - 89s 332ms/step - loss: 0.6329 - accuracy: 0.7901 - val_loss: 0

## Prediction on Test Set

In [13]:
model.load_weights('/kaggle/working/VGG19.h5')

In [15]:
test = []
for file in sample_submission['file']:
    test.append(os.path.join(test_dir,file))

test = pd.DataFrame(test, columns=['file'])
print(test.head(5))

                                                file
0  /kaggle/input/plant-seedlings-classification/t...
1  /kaggle/input/plant-seedlings-classification/t...
2  /kaggle/input/plant-seedlings-classification/t...
3  /kaggle/input/plant-seedlings-classification/t...
4  /kaggle/input/plant-seedlings-classification/t...


In [16]:
x_test = []

for i in range(len(test)):
    img = cv2.imread(test['file'][i])
    img = cv2.resize(img,(img_size,img_size))
    img_stack = segment_plant(img)
    img_stack = sharpen_image(img_stack)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    x_test.append(np.array(img))
    
x_test = np.array(x_test)

In [17]:
Pred_labels = np.argmax(model.predict(x_test),axis = 1)
Pred_labels = pd.DataFrame(Pred_labels,index=None,columns=['species_num'])

In [18]:
sample_submission['species'] = [labels[i] for i in Pred_labels['species_num']]
sample_submission

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Maize
2,007b3da8b.png,Maize
3,0086a6340.png,Maize
4,00c47e980.png,Maize
...,...,...
789,fea355851.png,Maize
790,fea3da57c.png,Maize
791,fef2ade8c.png,Maize
792,ff65bc002.png,Charlock


### Save File for Submission

In [19]:
sample_submission.to_csv('/kaggle/working/VGG19-submission.csv',index=False)

## Test model on New Test Dataset (images downloaded from Google)

In [20]:
test_new_dir = '/kaggle/input/plant-seedlings-test/'
Test_new = pd.read_csv('/kaggle/input/plant-seedlings-test/Test_new_actual.csv')

In [21]:
test_new = []
for file in Test_new['file']:
    test_new.append(os.path.join(test_new_dir,file))

test_new = pd.DataFrame(test_new, columns=['file'])
print(test_new.head(5))

                                               file
0  /kaggle/input/plant-seedlings-test/3c75e7d79.png
1        /kaggle/input/plant-seedlings-test/456.jpg
2        /kaggle/input/plant-seedlings-test/789.jpg
3        /kaggle/input/plant-seedlings-test/123.jpg
4     /kaggle/input/plant-seedlings-test/131415.jpg


In [22]:
x_test_new = []

for i in range(len(test_new)):
    img = cv2.imread(test_new['file'][i])
    img = cv2.resize(img,(img_size,img_size))
    img_stack = segment_plant(img)
    img_stack = sharpen_image(img_stack)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    x_test_new.append(np.array(img))
    
x_test_new = np.array(x_test_new)

In [23]:
Pred_labels_new = np.argmax(model.predict(x_test_new),axis = 1)
Pred_labels_new = pd.DataFrame(Pred_labels_new,index=None,columns=['species_num'])

In [24]:
Test_new['prediction'] = [labels[i] for i in Pred_labels_new['species_num']]
Test_new

,file,species,prediction
0,3c75e7d79.png,Charlock,Charlock
1,456.jpg,Fat Hen,Cleavers
2,789.jpg,Charlock,Charlock
3,123.jpg,Fat Hen,Fat Hen
4,131415.jpg,Small-flowered Cranesbill,Charlock
5,101112.jpg,Charlock,Common wheat
6,161718.jpg,Chickweed,Cleavers
7,da38229c7.png,Cleavers,Charlock
8,202122.jpg,Cleavers,Loose Silky-bent
9,0ebf8f2f4.png,Maize,Maize


### The VGG19 Model get 5/20 correct

In [ ]:
Test_new.to_csv('/kaggle/working/test_new_result_VGG19.csv',index=False)